# Insula API Data Access & Discovery

## Overview

This document serves as a quick guide of how to access Insula's API for various tasks in Python. This guide will show how to discover, filter and download data.

***Steps***:
1. Preparation
2. Catalog Search
3. Download API
4. WFS
5. WMS

## Preparation

These preparation blocks have to be executed as a prerequisite for every subsequent step to work correctly, as these variables will be used to configure each http request.
Be sure to put a valid `USERNAME/TOKEN` pair, and check that the domain inside `BASE_URL` is still valid.

In [ ]:
pip install InsulaWorkflowClient geopandas matplotlib rasterio mapclassify folium furl

In [ ]:
import requests
import base64
import json
import getpass

In [ ]:
USERNAME=getpass.getpass()

In [ ]:
PASSWORD=getpass.getpass()

In [ ]:
from InsulaWorkflowClient import InsulaOpenIDConnect

BASE_URL="https://insula.earth"

insulaAuth: InsulaOpenIDConnect = InsulaOpenIDConnect(
        authorization_endpoint="https://identity.insula.earth/realms/eopaas/protocol/openid-connect/auth",
        token_endpoint="https://identity.insula.earth/realms/eopaas/protocol/openid-connect/token",
        redirect_uri="http://localhost:9207/auth",
        client_id="api-client"
    )
insulaAuth.set_user_credentials(username=USERNAME, password=PASSWORD)

bearer = insulaAuth.get_authorization_header()
print(bearer)
HEADERS={'Authorization': bearer }

## Catalog Search
We can query the `parameters` endpoint to get a list of all availiable parameters to set:

In [ ]:
url=f'{BASE_URL}/secure/api/v2.0/search/parameters'
run_request=requests.get(url,headers=HEADERS, verify=True)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

Many of the sequent API usage share a similarity for what concerns the search. For example the page and the number of results to display are used in many other API calls. In this catalog search many other filters could be applied: catalog, mission, AOI, TOI, orbit number, etc.

In [ ]:
page=0
resultsPerPage=20
catalogue='SATELLITE'
mission='sentinel3'
aoi='POLYGON+((110.12588478529223+-6.885079244763438,+110.12588478529223+-7.29186930980306,+111.05481310696132+-7.29186930980306,+111.06289082671104+-6.472900901082527,+111.02519514358733+-6.402857492192213,+110.86094988721777+-6.394775588065212,+110.67247167702403+-6.451349019721208,+110.62131326460076+-6.664173077411836,+110.48130096205539+-6.739604353875493,+110.52976676699129+-6.7854019476178165,+110.41667982033255+-6.920100897825102,+110.29282275186166+-6.8958548771431385,+110.17973575384669+-6.804259792425478,+110.1204997757424+-6.882385002675806,+110.12588478529223+-6.885079244763438))'
start='2021-04-01T00:00:00.000Z'
end='2021-09-30T23:59:59.000Z'
s3proclevel='2'
orbitDirection='ascending'
s3instrument='SLSTR'
producttype='SL_2_LST___'

url=f'{BASE_URL}/secure/api/v2.0/search?page={page}&resultsPerPage={resultsPerPage}&catalogue={catalogue}&mission={mission}&aoi={aoi}&productDateStart={start}&productDateEnd={end}&s3ProcessingLevel={s3proclevel}&orbitDirection={orbitDirection}&s3Instrument={s3instrument}&productType={producttype}'
run_request=requests.get(url,headers=HEADERS, verify=True)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

Next are other simpler examples for querying the Reference Data and Output Products:

In [ ]:
catalogue='REF_DATA'
collection='eopaas9ced0b003cf840b9a1138a8e6609cc73' #Phisat2-Sat2Map
aoi='POLYGON+((-19.52667870858845+30.81101444071784,+24.67455049168501+30.81101444071784,+24.67455049168501+59.58309365984046,+-19.52667870858845+59.58309365984046,+-19.52667870858845+30.81101444071784))'


url=f'{BASE_URL}/secure/api/v2.0/search?page=0&resultsPerPage=20&catalogue={catalogue}&refDataCollection={collection}&aoi={aoi}'
run_request=requests.get(url,headers=HEADERS, verify=True)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

In [ ]:
catalogue='PLATFORM_PRODUCTS'
collection='eopaasafe146198f5d473f981ae2b05d699261' #defaultOutput-21

url=f'{BASE_URL}/secure/api/v2.0/search?page=0&resultsPerPage=20&catalogue={catalogue}&collection={collection}'
run_request=requests.get(url,headers=HEADERS, verify=True)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

## Download API
We will query again the Output Products but this time we're searching for a specific file: `LC_2017_Rice_Dry_Agri.zip`, feel free to change these parameters with other collections or products for further testing.

In [ ]:
catalogue='PLATFORM_PRODUCTS'
collection='eopaas5c1a14e8fd684fa69fb115887a1cfb00' #gda_agri_vector_layers
product='LC_2017_Rice_Dry_Agri.zip'

url=f'{BASE_URL}/secure/api/v2.0/search?page=0&resultsPerPage=20&catalogue={catalogue}&collection={collection}&identifier={product}'

run_request=requests.get(url,headers=HEADERS, verify=True)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

We searched for a specific filename so we should only have one result, we can access the links for this product inside this response. To use the download API simply call the download link as shown below.

In [ ]:
file=run_request_dict['features'][0]
#https://insula.earth/secure/api/v2.0/search/products/platform/47730
url=file['properties']['_links']['download']['href']

r = requests.get(url, headers=HEADERS, stream=True)
if r.status_code == 200:
    with open('gda_rice.zip', 'wb') as f:
        for chunk in r:
            f.write(chunk)

## WFS Visualization:
We will use the same product of the download section and expand on that: in the same section we can access the links for WMS and WFS services for this file. With the `furl` library we can extract the value of the layer parameter that will be used in the following snippets:

In [ ]:
from furl import furl

wfs_url=file['properties']['_links']['wfs']['href']
print("WFS: "+wfs_url)

wms_url=file['properties']['_links']['wms']['href']
print("WMS: "+wms_url)

layer=furl(wms_url).args['layers']
print("Layer: "+layer)

style=layer.replace(':','_')
print("Style: "+style)

cql_filter=furl(wms_url).args['cql_filter']
print("cql_filter: "+cql_filter)

To download and visualize a shapefile trough Insula's WFS interface we can use the GetFeature request.
The only thing to specify is the name of the product, we could declare a `bbox` parameter to narrow the area of interest.
The default format is GML, but we can switch to Shapefile or GeoJSON using the `outputFormat` parameter.
All the formats are known to `geopandas`, the library used to read and visualize these objects.

In [ ]:
url="https://geoserver.insula.earth/geoserver/wfs"
params = dict(
    version="2.0.0",
    request="GetFeature",
    typeName=layer,
    cql_filter=cql_filter,     # optional
    #outputFormat="shape-zip" or "application/json",
    #cql_filter=cql_filter,    # optional
    #srsName="CRS:84"          # optional
    #bbox="110.161869812,-7.454166667,111.824762701,-6.380393028"    # optional
)

r = requests.get(url, headers=HEADERS, params=params, stream=True)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

gdf = gpd.read_file(r.content)#.to_crs("epsg:4326")
print(gdf)

gdf.plot(
    column="Toponimi",
    legend=True
)
plt.show()

#or

gdf.explore(column='Toponimi')

## WMS Visualization:
We can use WMS to download and visualize a raster file or a raster projection of a shapefile by using the `GetMap` request.
Here we have more parameters to specify the name of the product in `layers` and optionally a style in the `styles` parameter.
In the `format` parameter we can specify `image/geotiff` or `image/png`,`image/jpeg` if we dont need georeferencing.
Declaring the aoi with a `bbox` parameter is mandatory, as the definition of the size of the output image (`width`,`height`).

One thing to note is that the returned picture shall have exactly the 
specified width and height in pixels. In the case where the aspect ratio of the BBOX and the ratio width/height ar 
different, the WMS shall stretch the returned map so that the resulting pixels could themselves be rendered in t e
aspect ratio of the BBOio.
Map distortions will be introduced if the aspect ratio WIDTH/HEIGHT is not commensurate with X, Y and the  ixel
ad maps.

In [ ]:
url="https://geoserver.insula.earth/geoserver/wms"
params = dict(
    version="1.3.0",
    request="GetMap",
    format="image/geotiff",   
    layers=layer,
    styles=style,         # Optional
    width="1672",
    height="1080",
    bbox="110.161869812,-7.454166667,111.824762701,-6.380393028",
    transparent="TRUE",   # Optional
    crs="CRS:84",         # Optional
    cql_filter=cql_filter # Optional
)

r = requests.get(url, headers=HEADERS, params=params, stream=True)
if r.status_code == 200:
    with open('gda_rice.tiff', 'wb') as f:
        for chunk in r:
            f.write(chunk)

In [ ]:
import rasterio
from rasterio.plot import show

with rasterio.open('gda_rice.tiff') as img:
    show(img)